---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    data = open('./datasets/university_towns.txt','r')
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    states = dict((v, k) for (k, v) in states.items())
    states1=[]
    region=[]
    for i in data:
        if "[edit]" in i:
            if i.find("[")>-1:
                x=i.find("[")
                a=i=i[:x]
            states1.append(a.strip())
            region.append(np.nan)
        else:
            if i.find("("):
                x=i.find("(")
                i=i[:x]
            states1.append(a.strip())
            region.append(i.strip())
    d={"RegionName":region,"State":states1}
    data = pd.DataFrame(d).dropna()
    data = data[["State","RegionName"]]
    data.reset_index(inplace=True)
    data.drop("index",axis=1,inplace=True)
    return data
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [22]:
def get_recession_start():
    recession = pd.read_excel("./datasets/gdplev.xls",header=5)
    recession = recession[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    recession.dropna(inplace=True)
    recession.reset_index(inplace=True)
    recession.drop("index",1,inplace=True)
    recession.rename(columns={"Unnamed: 4":"Period"},inplace=True)
    a=recession.loc[recession["Period"]== "2000q1"]
    a=a.index[0]
    recession= recession[a:]
    recession = recession.reset_index().drop("index",1)
    for i in range(len(recession["GDP in billions of chained 2009 dollars.1"])):
        if recession["GDP in billions of chained 2009 dollars.1"][i]>recession["GDP in billions of chained 2009 dollars.1"][i+1] and recession["GDP in billions of chained 2009 dollars.1"][i+1]>recession["GDP in billions of chained 2009 dollars.1"][i+2]:
            x=i+1
            break
    return recession["Period"][x]
get_recession_start()

'2008q3'

In [23]:
def get_recession_end():
    recession = pd.read_excel("./datasets/gdplev.xls",header=5)
    recession = recession[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    recession.dropna(inplace=True)
    recession.reset_index(inplace=True)
    recession.drop("index",1,inplace=True)
    recession.rename(columns={"Unnamed: 4":"Period"},inplace=True)
    a=recession.loc[recession["Period"]== get_recession_start()]
    a=a.index[0]
    recession= recession[a:]
    recession = recession.reset_index().drop("index",1)
    for i in range(len(recession["GDP in billions of chained 2009 dollars.1"])):
        if recession["GDP in billions of chained 2009 dollars.1"][i]<recession["GDP in billions of chained 2009 dollars.1"][i+1] and recession["GDP in billions of chained 2009 dollars.1"][i+1]<recession["GDP in billions of chained 2009 dollars.1"][i+2]:
            x=i+2
            break
    return recession["Period"][x]
get_recession_end()

'2009q4'

In [24]:
def get_recession_bottom():
    recession = pd.read_excel("./datasets/gdplev.xls",header=5)
    recession = recession[["Unnamed: 4","GDP in billions of chained 2009 dollars.1"]]
    recession.dropna(inplace=True)
    recession.reset_index(inplace=True)
    recession.drop("index",1,inplace=True)
    recession.rename(columns={"Unnamed: 4":"Period"},inplace=True)
    a=recession.loc[recession["Period"]== get_recession_start()]
    a=a.index[0]
    b=recession.loc[recession["Period"]== get_recession_end()]
    b=b.index[0]
    recession = recession[a:b+1]
    return recession["Period"][recession["GDP in billions of chained 2009 dollars.1"].idxmin()]
get_recession_bottom()

'2009q2'

In [14]:
def convert_housing_data_to_quarters():
    data = pd.read_csv("./datasets/City_Zhvi_AllHomes.csv")
    data["State"] = data["State"].replace(states)
    data.set_index(["State","RegionName"],inplace=True)
    columns_to_keep=[]
    for i in range(0,17):
        for j in range(1,13):
            a= str(2000+i) +"-0" +str(j)
            if j>=10:
                a= str(2000+i) +"-" +str(j)
            columns_to_keep.append(a)
    remove=['2016-09','2016-10','2016-11','2016-12']
    for x in remove:
        columns_to_keep.remove(x)
    data = data[columns_to_keep]
    x=0
    for i in range(0,17):
        for j in range(1,5):
            if data.columns[x+2] == "2000q1":
                data[str(2000+i)+"q"+str(j)]=data[[data.columns[x],data.columns[x+1]]].mean(axis=1)
                #print(str(2000+i)+"q"+str(j),"  ",data.columns[x],"  ",data.columns[x+1])
                break
            data[str(2000+i)+"q"+str(j)]=data[[data.columns[x],data.columns[x+1],data.columns[x+2]]].mean(axis=1)
            #print(str(2000+i)+"q"+str(j),"  ",data.columns[x],"  ",data.columns[x+1],"   ",data.columns[x+2])
            x=x+3
    for i in columns_to_keep:
        data.drop(i,1,inplace=True)
    return data
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [25]:
def run_ttest():
    towns = get_list_of_university_towns()
    startdate = get_recession_start()
    bottomdate = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    houses = houses.reset_index()
    houses['recession_diff'] = houses[startdate] - houses[bottomdate]
    towns_houses = pd.merge(houses, towns, how='inner', on=['State', 'RegionName'])
    towns_houses['ctown'] = True
    houses = pd.merge(houses, towns_houses, how='outer', on = ['State', 'RegionName',
                                                              bottomdate, startdate, 
                                                              'recession_diff'])
    houses['ctown'] = houses['ctown'].fillna(False)
    unitowns = houses[houses['ctown'] == True]
    not_unitowns = houses[houses['ctown'] == False]
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    different = True if p < 0.01 else False
    betters = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"

    
    return different,p,betters
run_ttest()

(True, 0.004325214853511201, 'university town')